# Segmentation

<br />

I will do segmentation for a medical dataset, "TGS Salt Identification Challenge" on Kaggle.


https://www.kaggle.com/c/tgs-salt-identification-challenge

<br />

In this task, I am going to use U-Net as segmentation method.

<br />

"U-Net: Convolutional Networks for Biomedical Image Segmentation"


Olaf Ronneberger, Philipp Fischer, and Thomas Brox


https://arxiv.org/pdf/1505.04597.pdf

## [Task 1] Fit and Predict

<br />

I fit and predict the dataset by using the following implementation of Keras.


https://github.com/zhixuhao/unet

## [Task 2] Code Reading